In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
df=pd.read_csv('datosVino.txt', sep='\t')
df.head()

,calidad,aroma,cuerpo,sabor,fuerza
0,9.8,3.3,2.8,3.1,4.1
1,12.6,4.4,4.9,3.5,3.9
2,11.9,3.9,5.3,4.8,4.7
3,11.1,3.9,2.6,3.1,3.6
4,13.3,5.6,5.1,5.5,5.1


In [ ]:
df[df.duplicated]

,calidad,aroma,cuerpo,sabor,fuerza


In [ ]:
df.isnull().sum()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   calidad  38 non-null     float64
 1   aroma    38 non-null     float64
 2   cuerpo   38 non-null     float64
 3   sabor    38 non-null     float64
 4   fuerza   38 non-null     float64
dtypes: float64(5)
memory usage: 1.6 KB


In [ ]:
y=df['calidad']
X=df.drop('calidad', axis=1)
regresion=LinearRegression().fit(X,y)
y_pred=regresion.predict(X)
print(r2_score(y, y_pred))
print(mean_squared_error(y,y_pred))

0.7059681622379318
1.1858198132902908


In [ ]:
y_pred=regresion.predict(X)
df['y_pred']=y_pred
residuales=y-y_pred
df['residuales']=residuales
df.head()

,calidad,aroma,cuerpo,sabor,fuerza,y_pred,residuales
0,9.8,3.3,2.8,3.1,4.1,9.550593,0.249407
1,12.6,4.4,4.9,3.5,3.9,10.974741,1.625259
2,11.9,3.9,5.3,4.8,4.7,11.719974,0.180026
3,11.1,3.9,2.6,3.1,3.6,10.182308,0.917692
4,13.3,5.6,5.1,5.5,5.1,13.240222,0.059778


# VALORES ATIPICOS

In [ ]:
MSE=np.mean(df['residuales']**2)
print(MSE)
MSE2=sum(df['residuales']**2)/df['calidad'].count()
print(MSE2)
df['standarized residuals']=df['residuales']/(MSE**0.5)
df.head()

1.1858198132902908
1.1858198132902908


,calidad,aroma,cuerpo,sabor,fuerza,y_pred,residuales,standarized residuals
0,9.8,3.3,2.8,3.1,4.1,9.550593,0.249407,0.229033
1,12.6,4.4,4.9,3.5,3.9,10.974741,1.625259,1.492496
2,11.9,3.9,5.3,4.8,4.7,11.719974,0.180026,0.165320
3,11.1,3.9,2.6,3.1,3.6,10.182308,0.917692,0.842728
4,13.3,5.6,5.1,5.5,5.1,13.240222,0.059778,0.054895


In [ ]:
df[df['standarized residuals']>3]

,calidad,aroma,cuerpo,sabor,fuerza,y_pred,residuales,standarized residuals


# NO HAY VALORES ATIPICOS

# PUNTOS DE BALANCEO

In [ ]:
hii=np.dot(X, np.dot(np.linalg.inv(np.dot(X.T,X)),X.T))
leverage=np.diag(hii)
df['hii']=leverage
df.head()

,calidad,aroma,cuerpo,sabor,fuerza,y_pred,residuales,standarized residuals,hii
0,9.8,3.3,2.8,3.1,4.1,9.550593,0.249407,0.229033,0.085794
1,12.6,4.4,4.9,3.5,3.9,10.974741,1.625259,1.492496,0.127304
2,11.9,3.9,5.3,4.8,4.7,11.719974,0.180026,0.165320,0.114926
3,11.1,3.9,2.6,3.1,3.6,10.182308,0.917692,0.842728,0.085936
4,13.3,5.6,5.1,5.5,5.1,13.240222,0.059778,0.054895,0.042706


In [ ]:
H=(2*(4+1))/(df['calidad'].count())
print(H)

0.2631578947368421


In [ ]:
df[df['hii']>H]

,calidad,aroma,cuerpo,sabor,fuerza,y_pred,residuales,standarized residuals,hii
13,16.1,7.7,6.6,6.7,3.7,16.825037,-0.725037,-0.665811,0.271601
36,10.3,4.3,5.5,3.5,5.8,9.822123,0.477877,0.438840,0.280610


In [ ]:
X.describe()

,aroma,cuerpo,sabor,fuerza
count,38.000000,38.000000,38.000000,38.000000
mean,4.847368,4.684211,4.768421,4.255263
std,1.082474,0.824138,1.028014,0.738039
min,3.300000,2.600000,2.900000,2.900000
25%,4.125000,4.150000,4.225000,3.700000
50%,4.650000,4.750000,4.800000,4.100000
75%,5.450000,5.375000,5.500000,4.775000
max,7.700000,6.600000,7.000000,6.000000


# HAY DOS PUNTOS DE BALANCEO (OBSERVACIONES 13 Y 36)

# PUNTOS INFLUYENTES

In [ ]:
import statsmodels.api as sm

# Ajusta tu modelo de regresión lineal
regresion2 = sm.OLS(y,X).fit()

# Calcula DFFITS
infl = regresion2.get_influence()
dffits = infl.dffits

In [ ]:
dffits

(array([ 0.38259241,  0.5657289 , -0.07446114,  0.65186346, -0.11297599,
         0.0489142 ,  0.08105856, -0.01607368,  0.55785199,  0.4224385 ,
         0.39975084,  0.4428242 , -0.07414006, -0.8106079 ,  0.27887006,
         0.17797276, -0.01365931, -0.00260539,  0.02444501, -0.68057783,
        -0.21896021, -0.04511179, -0.2115103 , -0.12197794, -0.30505316,
        -0.05482875, -0.01319581, -0.17093888, -0.24067114, -0.15912103,
        -0.71817162, -0.05893434,  0.63711606, -0.1332223 , -0.09393402,
         0.34692524, -0.43836764, -0.1046025 ]),
 0.6488856845230502)

In [ ]:
df['DFFITS']=dffits[0]

In [ ]:
df.head()

,calidad,aroma,cuerpo,sabor,fuerza,y_pred,residuales,standarized residuals,hii,DFFITS
0,9.8,3.3,2.8,3.1,4.1,9.550593,0.249407,0.229033,0.085794,0.382592
1,12.6,4.4,4.9,3.5,3.9,10.974741,1.625259,1.492496,0.127304,0.565729
2,11.9,3.9,5.3,4.8,4.7,11.719974,0.180026,0.165320,0.114926,-0.074461
3,11.1,3.9,2.6,3.1,3.6,10.182308,0.917692,0.842728,0.085936,0.651863
4,13.3,5.6,5.1,5.5,5.1,13.240222,0.059778,0.054895,0.042706,-0.112976


In [ ]:
VALUE=2*(((4+1)/(df['calidad'].count()))**0.5)
VALUE

0.7254762501100116

In [ ]:
df[df['DFFITS'].abs()>VALUE]

,calidad,aroma,cuerpo,sabor,fuerza,y_pred,residuales,standarized residuals,hii,DFFITS
13,16.1,7.7,6.6,6.7,3.7,16.825037,-0.725037,-0.665811,0.271601,-0.810608


HAY UN PUNTO INFLUYENTE (13) PARA DFFITS

In [ ]:
cooks_distance = infl.cooks_distance
cooks_distance

(array([3.60015163e-02, 7.72977647e-02, 1.42627344e-03, 9.62658696e-02,
        3.25932899e-03, 6.15086689e-04, 1.68652537e-03, 6.65434599e-05,
        6.89844022e-02, 4.14791863e-02, 3.72487392e-02, 4.93113276e-02,
        1.41470507e-03, 1.60669367e-01, 1.98800982e-02, 8.00082085e-03,
        4.80558593e-05, 1.74843187e-06, 1.53845355e-04, 1.11396716e-01,
        1.21721625e-02, 5.23675395e-04, 1.12410496e-02, 3.76707217e-03,
        2.25189848e-02, 7.73254748e-04, 4.48504490e-05, 7.38897709e-03,
        1.46811340e-02, 6.49136813e-03, 1.23077612e-01, 8.94072851e-04,
        9.69938313e-02, 4.55308803e-03, 2.26358644e-03, 2.98762568e-02,
        4.87692851e-02, 2.80861786e-03]),
 array([0.99739798, 0.98871009, 0.9999957 , 0.98296603, 0.99997761,
        0.9999992 , 0.99999399, 0.99999999, 0.99089812, 0.99657373,
        0.99721969, 0.99521316, 0.99999577, 0.95674743, 0.99918743,
        0.99986605, 1.        , 1.        , 0.99999995, 0.97768482,
        0.99969188, 0.99999942, 0.9997

In [ ]:
df['Cook D']=cooks_distance[0]
df.head()

,calidad,aroma,cuerpo,sabor,fuerza,y_pred,residuales,standarized residuals,hii,DFFITS,Cook D
0,9.8,3.3,2.8,3.1,4.1,9.550593,0.249407,0.229033,0.085794,0.382592,0.036002
1,12.6,4.4,4.9,3.5,3.9,10.974741,1.625259,1.492496,0.127304,0.565729,0.077298
2,11.9,3.9,5.3,4.8,4.7,11.719974,0.180026,0.165320,0.114926,-0.074461,0.001426
3,11.1,3.9,2.6,3.1,3.6,10.182308,0.917692,0.842728,0.085936,0.651863,0.096266
4,13.3,5.6,5.1,5.5,5.1,13.240222,0.059778,0.054895,0.042706,-0.112976,0.003259


In [ ]:
df[df['Cook D']>1]

,calidad,aroma,cuerpo,sabor,fuerza,y_pred,residuales,standarized residuals,hii,DFFITS,Cook D


NOY HAY PUNTOS INFLUYENTES DISTANCIA DE COOK

SE PROCEDE A ELIMINAR LAS OBSERVACIONES 13 Y 16

In [ ]:
df = df.drop([13, 16]).reset_index(drop=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   calidad                36 non-null     float64
 1   aroma                  36 non-null     float64
 2   cuerpo                 36 non-null     float64
 3   sabor                  36 non-null     float64
 4   fuerza                 36 non-null     float64
 5   y_pred                 36 non-null     float64
 6   residuales             36 non-null     float64
 7   standarized residuals  36 non-null     float64
 8   hii                    36 non-null     float64
 9   DFFITS                 36 non-null     float64
 10  Cook D                 36 non-null     float64
dtypes: float64(11)
memory usage: 3.2 KB


# MULTICOLINEALIDAD

In [ ]:
orden=[['aroma', 'cuerpo', 'sabor', 'fuerza'],
       ['cuerpo', 'aroma', 'sabor', 'fuerza'],
       ['sabor', 'aroma', 'cuerpo', 'fuerza'],
       ['fuerza', 'aroma', 'cuerpo', 'sabor']]

In [ ]:
FIV=[]

for i in orden:
  Xo=i[1:]
  yo=i[0]
  Xoo=df[Xo]
  yoo=df[f'{yo}']
  regresionm=LinearRegression().fit(Xoo,yoo)
  y_predm=regresionm.predict(Xoo)
  FIVc=1/(1-r2_score(yoo,y_predm))
  FIV.append(FIVc)

FIV

[2.160093921713764, 1.5952644394225126, 2.694564232317327, 1.095941267067689]

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

XL = X[['aroma', 'cuerpo', 'sabor', 'fuerza']]
XL['1']=1
XL = XL[['1','aroma', 'cuerpo', 'sabor', 'fuerza']]

vif_data = pd.DataFrame()
vif_data["feature"] = XL.columns

vif_data["VIF"] = [variance_inflation_factor(XL.values, i)
                          for i in range(len(XL.columns))]

print(vif_data)

  feature        VIF
0       1  61.742115
1   aroma   2.160094
2  cuerpo   1.595264
3   sabor   2.694564
4  fuerza   1.095941


En general, se considera que un valor de VIF inferior a 10 indica que la multicolinealidad no es un problema significativo en el modelo de regresión. Esto significa que las variables predictoras están relativamente poco correlacionadas entre sí, lo que es deseable para evitar problemas de interpretación de los coeficientes de regresión y la precisión de las predicciones.

In [ ]:
XM=X
XM=np.array(XM[['aroma', 'cuerpo', 'sabor', 'fuerza']])

In [ ]:
MAT=np.dot(XM.T,XM)
VPROPIOS=np.linalg.eigvals(MAT)
VPROPIOS

array([3354.76726679,   27.47517982,   14.82685867,    9.46069472])

In [ ]:
K=np.max(VPROPIOS)/np.min(VPROPIOS)
K

354.6005198219305

K=354.6, esto caería en la categoría de "multicolinealidad moderada", sin embargo, estamos lejos de alcanzar el umbral de 1000
κ>1000, entonces la multicolinealidad no se considera grave y no es tan preocupante.

# SELECCION DE VARIABLES

In [ ]:
import itertools

variables = ['aroma', 'cuerpo', 'sabor', 'fuerza']
combinaciones = []

for r in range(1, len(variables) + 1):
    combinaciones.extend(itertools.combinations(variables, r))

combinaciones.append(tuple(variables))

combinaciones_finales = sorted(set(combinaciones))

for i in combinaciones_finales:
  print(i)

('aroma',)
('aroma', 'cuerpo')
('aroma', 'cuerpo', 'fuerza')
('aroma', 'cuerpo', 'sabor')
('aroma', 'cuerpo', 'sabor', 'fuerza')
('aroma', 'fuerza')
('aroma', 'sabor')
('aroma', 'sabor', 'fuerza')
('cuerpo',)
('cuerpo', 'fuerza')
('cuerpo', 'sabor')
('cuerpo', 'sabor', 'fuerza')
('fuerza',)
('sabor',)
('sabor', 'fuerza')


In [ ]:
MSEmod=[]
R2_AjMod=[]
R2=[]
p=[]
n=df['calidad'].count()

y=df['calidad']

for i in combinaciones_finales:
  Xcom = df[list(i)]
  RegresionCom = LinearRegression().fit(Xcom, y)
  y_predCom=RegresionCom.predict(Xcom)
  pc=len(i)
  p.append(pc)
  MSEmod.append((mean_squared_error(y,y_predCom)))
  R2n=r2_score(y,y_predCom)
  R2.append(R2n)
  R2_AjMod.append((1-(((1 - R2n) * (n - 1)) / (n - pc - 1))))


In [ ]:
RESULT=pd.DataFrame()
RESULT['Modelo']=combinaciones_finales
RESULT['R2 Ajustado']=R2_AjMod
RESULT['R2']=R2
RESULT['MSE']=MSEmod
RESULT['p']=p
RESULT['n']=n

In [ ]:
RESULT

,Modelo,R2 Ajustado,R2,MSE,p,n
0,"(aroma,)",0.433575,0.449759,2.099012,1,36
1,"(aroma, cuerpo)",0.463426,0.494088,1.929911,2,36
2,"(aroma, cuerpo, fuerza)",0.509558,0.551596,1.710534,3,36
3,"(aroma, cuerpo, sabor)",0.586941,0.622346,1.440641,3,36
4,"(aroma, cuerpo, sabor, fuerza)",0.637931,0.679310,1.223340,4,36
5,"(aroma, fuerza)",0.466270,0.496769,1.919683,2,36
6,"(aroma, sabor)",0.599128,0.622035,1.441829,2,36
7,"(aroma, sabor, fuerza)",0.647523,0.677736,1.229346,3,36
8,"(cuerpo,)",0.208734,0.231342,2.932213,1,36
9,"(cuerpo, fuerza)",0.201384,0.247019,2.872407,2,36


In [ ]:
RESULT.sort_values(by='MSE',ascending=True)

,Modelo,R2 Ajustado,R2,MSE,p,n
4,"(aroma, cuerpo, sabor, fuerza)",0.637931,0.679310,1.223340,4,36
7,"(aroma, sabor, fuerza)",0.647523,0.677736,1.229346,3,36
11,"(cuerpo, sabor, fuerza)",0.605493,0.639308,1.375937,3,36
14,"(sabor, fuerza)",0.616147,0.638082,1.380614,2,36
3,"(aroma, cuerpo, sabor)",0.586941,0.622346,1.440641,3,36
6,"(aroma, sabor)",0.599128,0.622035,1.441829,2,36
10,"(cuerpo, sabor)",0.571445,0.595934,1.541398,2,36
13,"(sabor,)",0.583775,0.595667,1.542416,1,36
2,"(aroma, cuerpo, fuerza)",0.509558,0.551596,1.710534,3,36
5,"(aroma, fuerza)",0.466270,0.496769,1.919683,2,36


SE ESCOGE EL SEGUNDO MODELO (aroma, sabor, fuerza) YA QUE, A PESAR DE SER EL SEGUNDO MEJOR EN TERMINOS DE MSE Y R2, SE SACRIFICA POCO RESPECTO AL PRIMER MODELO Y SE PUEDE ELIMINAR LA VARIABLE (cuerpo).

> Añadir blockquote

